#### **2D Numerical Simulation of Soil Water Infiltration in Loam Soil**
#### **Purpose:** This project implements a Finite Difference numerical solver to model the time-dependent movement of moisture in unsaturated porous media. By simulating the diffusion of water from a surface drip emitter, the study visualizes the formation of wetting bulbs and analyzes the spatial distribution of hydraulic potential within the root zone.

##### **Author:** Bello Oluwatobi

##### **Last Updated:** November 20, 2025

### #1 Importing libraries

In [ ]:
# importing neccessary libraries
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from matplotlib.animation import PillowWriter

### #2 Defining study parameters

In [ ]:
# setting the dimensions of the root zone area - 50cm x 50cm 
Lx, Ly = 0.5, 0.5       # in meters

# setting the grid resolution
nx, ny = 50, 50         # 40x40 grid point

# defining the soil hydraulic diffusivity (D) in m^2/hour
D = 0.002               # Loam soil properties

# defining the total duration of the simulation
total_time = 24.0       # simulating 24 hours of water infiltration

# setting the frame rate for the video output of the simulation
fps = 15

In [ ]:
# calculating the physical distance between two horizontal grid points (dx)
dx = Lx / (nx - 1)

# calculating the physical distance between two vertical grid points (dy)
dy = Ly / (ny - 1)

# calculating the time step (dt) using the Von Neumann stability criterion
dt = (dx**2 * dy**2) / (2 * D * (dx**2 + dy**2)) * 0.9 # multiplying by 0.9 for a 10% safety margin

In [ ]:
# setting the time step for simulation frame capture - 10 simulation minutes (0.1667 hours)
sim_steps_per_frame = int(0.1667 * (1.0/dt)) 

# calculating the total number of frames needed for the full 24-hour simulation
total_frames = int(total_time / (sim_steps_per_frame * dt))

# Print status to console so the user knows the simulation size
print(f"Total no. of frames to be generated for {total_time} hours of simulation = {total_frames}")

### #3 Initializing the soil grid

In [ ]:
# creating a 2D array to represent the soil grid
theta = np.zeros((ny, nx)) + 0.1 # initializing soil moisture to 10% (dry condition)

# finding the index of the middle column to place the emitter
mid_x = nx // 2

In [ ]:
# creating the initial soil heatmap plot
fig, ax = plt.subplots(figsize=(8, 8))

im = ax.imshow(theta, cmap='Spectral', interpolation='bicubic', 
               extent=[0, Lx, Ly, 0], vmin=0, vmax=1)

# adding a colorbar to the side (0% to 100% wet)
cbar = plt.colorbar(im, ax=ax)
cbar.set_label('Soil Moisture Saturation (0=Dry, 1=Wet)')

# setting the title and axis labels
ax.set_title("Initial Soil Heatmap (50cm x 50cm) @ t = 0.0 hours")
ax.set_xlabel("Width (m)")
ax.set_ylabel("Depth (m)")

# Create subtle grid lines based on your dx/dy
ax.set_xticks(np.linspace(0, Lx, nx), minor=True)
ax.set_yticks(np.linspace(0, Ly, ny), minor=True)
ax.grid(which='minor', color='black', linestyle='-', linewidth=1, alpha=0.5)

# saving initial soil plot figure
plt.savefig("../results/soil_plot_initial.png", dpi=300)


# initializing a variable to hold the white contour lines
global_contours = None

### #4 Defining simulation function

In [ ]:
# defining function to update the simulation plots
def update(frame):
    # accessing the global variables 'theta' (soil) and 'global_contours' (lines)
    global theta, global_contours
    
    # running loop to perform calculation for every simulation frame
    for _ in range(sim_steps_per_frame):
        
        # setting boundary condition - drip emitter releasing water constantly (100% wet)
        theta[0, mid_x-1 : mid_x+2] = 1.0 
        

        # getting the neighbours for the cells
        tc = theta[1:-1, 1:-1]  # center cells (skipping the walls)
        tt = theta[0:-2, 1:-1]  # top neighbors
        tb = theta[2:, 1:-1]    # bottom neighbors
        tl = theta[1:-1, 0:-2]  # left neighbors
        tr = theta[1:-1, 2:]    # right neighbors
        
        # applying the discretized 2D diffusion equation
        theta[1:-1, 1:-1] = tc + D * dt * (
            (tr - 2*tc + tl) / dx**2 +  # diffusion in X direction
            (tb - 2*tc + tt) / dy**2    # diffusion in Y direction
        )
        
        # setting boundary condition - Neumann boundary conditions for the walls
        theta[:, 0]  = theta[:, 1]   # Left wall copies value from its right neighbor
        theta[:, -1] = theta[:, -2]  # Right wall copies value from its left neighbor
        theta[-1, :] = theta[-2, :]  # Bottom wall copies value from above neighbor

    
    # updating the heatmap image with the new moisture data
    im.set_data(theta)
    
    # checking if the old contour lines exist from the previous frame
    if global_contours is not None:
        # removing the old contour
        global_contours.remove()
            
    # drawing new white contour lines at specific moisture levels (20%, 40%, 60%, 80%)
    global_contours = ax.contour(theta, levels=[0.2, 0.4, 0.6, 0.8], 
                                 colors='white', linewidths=1, alpha=0.5,
                                 extent=[0, Lx, Ly, 0], origin='upper')
    
    # updating the title to show the current simulation time in hours
    current_time = (frame * sim_steps_per_frame * dt)
    ax.set_title(f"Soil Water Infiltration - Simulation Time: {current_time:.1f} Hours")
    
    return im,

### #5 Running the simulation

In [ ]:
# creating the animation object using the Figure and simulation function
ani = animation.FuncAnimation(fig, update, frames=total_frames, blit=False)

# initializing the GIF writer with the desired frame rate
writer = PillowWriter(fps=fps)

# saving the final animation
ani.save("../results/soil_moisture_timelapse.gif", writer=writer)

# print confirmation when finished
print("Simulation done and animation saved...")

plt.close()